# Training the model on the full data set
to make every wine accessible, we train the model again on the full dataset

In [1]:
from IPython.display import display, clear_output
import os
import seaborn as sns
import numpy as np
from importlib import reload
# import random
# import pickle
# import time
# import matplotlib.pyplot as plt
# import itertools
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics.pairwise import cosine_similarity, cosine_distances
# from sklearn.metrics import confusion_matrix

# from scipy.spatial.distance import cosine
# from nltk.corpus import words
# import gensim

from gensim.models.doc2vec import Doc2Vec, TaggedDocument
# from nltk.tokenize import word_tokenize

# from collections import Counter
%matplotlib inline
from wine_and_cheese_utils import WineList, df_columns, regexp_tokenizer

import warnings
    
warnings.filterwarnings("ignore")

In [2]:
wl = WineList(file='cleaned')


In [6]:
# collect the tagged document object
tagged_data = [TaggedDocument(words=wl.tokenize(row.description), tags=[row.Index]) for row in wl.df.itertuples()]

In [4]:
retrain = False

column_name = 'region_1'
# load or train the model
model_file_name = 'doc2vec_on_{}_no_region_variety_full_dataset.model'.format(column_name)
if not os.path.exists(model_file_name) or retrain:
    max_epochs = 20
    vec_size = 40
    alpha = 0.025

    model = Doc2Vec(vector_size=vec_size,
                    alpha=alpha, 
    #                 min_alpha=0.00025,
                    min_count=10,
                    dm =0)

    model.build_vocab(tagged_data)

    for epoch in range(max_epochs):
        clear_output(wait=True)
        display('iteration {0}'.format(epoch))
        model.train(tagged_data,
                    total_examples=model.corpus_count,
                    epochs=model.epochs)
        # decrease the learning rate
        model.alpha -= 0.0002
        # fix the learning rate, no decay
        model.min_alpha = model.alpha

    model.save(model_file_name)
else:
    model = Doc2Vec.load(model_file_name)

In [6]:
wl.get_wines_from_desc('strong cherry tannic win',model,topn=10)


([32152, 84918, 104932, 80979, 68902, 121627, 97114, 24113, 45877, 64019],
 ['strong', 'cherry', 'tannic', 'win'])

In [14]:
wl.get_wines_from_desc('sour cherry and strong tannins',model,topn=5)


([79898, 97111, 51420, 110420, 35233], ['sour', 'cherry', 'strong', 'tannins'])

In [26]:
model.wv.most_similar("everyday")

[('gaillac', 0.5281097888946533),
 ('shape', 0.5210387110710144),
 ('presently', 0.5131081342697144),
 ('argentinean', 0.5063772797584534),
 ('board', 0.5056312084197998),
 ('respect', 0.5022346377372742),
 ('softer', 0.4965851306915283),
 ('sensuous', 0.47983869910240173),
 ('kirsch', 0.4779838025569916),
 ('dominated', 0.47423386573791504)]

In [32]:
a = set([1,3,4,43])
a.add(6)

In [33]:
a

{1, 3, 4, 6, 43}

In [35]:
%timeit 'chill' in model.wv.vocab

115 ns ± 13.6 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
